In [ ]:
import cupy
import math
from numba import cuda
import numba
import numpy

import pyibm

In [ ]:
pyibm.__version__

In [ ]:
config = dict(x=dict(start=-2.0, end=2.0, num_cells=20),
              y=dict(start=-2.0, end=2.0, num_cells=20))
grid = pyibm.GridBase(config=config)
grid

In [ ]:
gridc = pyibm.GridCellCentered(grid=grid)
gridx = pyibm.GridFaceX(grid=grid)
gridy = pyibm.GridFaceY(grid=grid)

In [ ]:
class EulerianField(object):
    """Eulerian field variable."""

    def __init__(self, grid=pyibm.GridBase(), ic=0.0, bc=None):
        """Initialize the field."""
        self.grid = grid
        self.size = grid.size
        self.shape = grid.shape
        if grid.size > 0:
            self.bc = bc
            shape = tuple(s + 2 for s in grid.shape)
            self.values = ic * numpy.ones(shape)

In [ ]:
ux = EulerianField(grid=gridx, ic=1.0)
uy = EulerianField(grid=gridy, ic=0.0)

In [ ]:
threads = (16, 16)
blocks = tuple(math.ceil(s / b) for s, b in zip(ux.shape, threads))

In [ ]:
@cuda.jit
def _convective2d_x(ux, uy, dx, dy, out):
    i, j = cuda.grid(2)
    n, m = ux.shape
    if 1 <= i < m - 1 and 1 <= j < m - 1:
        hxx = (((ux[j, i + 1] + ux[j, i]) / 2)**2 -
               ((ux[j, i] + ux[j, i - 1]) / 2)**2) / dx[i]
        hxy = ((ux[j, i] + ux[j + 1, i]) / 2 *
               (uy[j, i] + uy[j , i + 1]) / 2 -
               (ux[j - 1, i] + ux[j, i]) / 2 *
               (uy[j - 1, i] + uy[j - 1, i + 1]) / 2) / dy[j]
        out[j, i] = hxx + hxy

In [ ]:
def convective_x(ux, uy):
    out = cupy.empty(ux.values.shape)
    dx = ux.grid.x.get_widths()
    dy = ux.grid.y.get_widths()
    _convective2d_x[blocks, threads](ux.values, uy.values,
                                     dx, dy, out)
    return cupy.asnumpy(out)

In [ ]:
@cuda.jit
def _convective2d_y(ux, uy, dx, dy, out):
    i, j = cuda.grid(2)
    n, m = uy.shape
    if 1 <= i < m - 1 and 1 <= j < m - 1:
        hyx = ((ux[j, i] + ux[j + 1, i]) / 2 *
               (uy[j, i] + uy[j, i + 1]) / 2 -
               (ux[j, i - 1] + ux[j + 1, i - 1]) / 2 *
               (uy[j, i - 1] + uy[j, i]) / 2) / dx[i]
        hyy = (((uy[j, i] + uy[j + 1, i]) / 2)**2 -
               ((uy[j - 1, i] + uy[j, i]) / 2)**2) / dy[j]
        out[j, i] = hyx + hyy

In [ ]:
def convective_y(ux, uy):
    out = cupy.empty(uy.values.shape)
    dx = uy.grid.x.get_widths()
    dy = uy.grid.y.get_widths()
    _convective2d_y[blocks, threads](ux.values, uy.values,
                                     dx, dy, out)
    return cupy.asnumpy(out)

In [ ]:
Hx = convective_x(ux, uy)
Hx.shape

In [ ]:
Hy = convective_y(ux, uy)
Hy.shape